In [11]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os



In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))

nb_train_samples = 18047
nb_val_samples = 4377
image_size = (224,224)
batch_size = 128
class_num = 10

In [5]:
def build_simple_convnet(input_shape=(224,224,3),class_num=10):
    
    model = Sequential()
#     model.add(Conv2D(32,(3,3),input_shape=input_shape,padding='valid',activation='relu'))
#     model.add(MaxPool2D())
    
#     model.add(Conv2D(32,(3,3),activation='relu'))
#     model.add(MaxPool2D())
    
#     model.add(Conv2D(64,(3,3),activation='relu'))
#     model.add(MaxPool2D()) 
    
#     model.add(Flatten())
#     model.add(Dense(256,activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(class_num,activation='softmax'))
    
#     model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(class_num, activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    
    return model

In [3]:
def generate_batches_data(dir_path, 
                    gen=image.ImageDataGenerator(), 
                    target_size=(224,224), 
                    batch_size=128, 
                    class_mode='categorical', 
                    shuffle=False):
    
    return gen.flow_from_directory(dir_path, 
                                   target_size=target_size, 
                                   batch_size=batch_size,
                                   class_mode=class_mode, 
                                   shuffle=shuffle)

In [4]:
def one_hot_encode(x):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
    """     
    return np.eye(10)[x]

In [5]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    
    return img


In [6]:
def test_model(model,test_path=test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

### try a simple CNN model

In [7]:

train_datagen = image.ImageDataGenerator(rescale=1./255,
                                         shear_range=0.2,
                                         rotation_range=10,
                                         height_shift_range=0.1,
                                         width_shift_range=0.1,
                                         channel_shift_range=5,
                                         horizontal_flip=True,
                                         fill_mode='nearest')

validation_datagen = image.ImageDataGenerator(rescale=1./255)

train_generator = generate_batches_data(
                                    dir_path=train_path,
                                    gen=train_datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = generate_batches_data(
                                    dir_path=validation_path,
                                    gen=validation_datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)


Found 18047 images belonging to 10 classes.
Found 4377 images belonging to 10 classes.


In [8]:
model = build_simple_convnet(input_shape=(224,224,3))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 43264)             0         
__________

In [10]:
model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=15,
                    callbacks=[TensorBoard()],
                    validation_data=validation_generator,
                    validation_steps=nb_val_samples // batch_size)

model.save('simple_cnn_model.h5')

Epoch 1/15
140/140 [==============================] - 311s 2s/step - loss: 1.8494 - acc: 0.3267 - val_loss: 2.3810 - val_acc: 0.3302
Epoch 2/15
140/140 [==============================] - 298s 2s/step - loss: 1.4382 - acc: 0.4699 - val_loss: 2.6039 - val_acc: 0.3888
Epoch 3/15
140/140 [==============================] - 295s 2s/step - loss: 1.2361 - acc: 0.5405 - val_loss: 2.5944 - val_acc: 0.3768
Epoch 4/15
140/140 [==============================] - 293s 2s/step - loss: 1.0975 - acc: 0.6004 - val_loss: 2.6119 - val_acc: 0.3690
Epoch 5/15
140/140 [==============================] - 292s 2s/step - loss: 0.9792 - acc: 0.6461 - val_loss: 3.2759 - val_acc: 0.3262
Epoch 6/15
140/140 [==============================] - 289s 2s/step - loss: 0.8837 - acc: 0.6807 - val_loss: 3.0090 - val_acc: 0.3714
Epoch 7/15
  6/140 [>.............................] - ETA: 9s - loss: 0.8075 - acc: 0.7240

KeyboardInterrupt: 

### try vgg16 model

In [35]:
#load VGG16 model
vgg16_base_model = VGG16(weights='imagenet', include_top=False)
vgg16_base_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [15]:
datagen = image.ImageDataGenerator(rescale=1./255)
train_generator = generate_batches_data(
                                    dir_path=train_path,
                                    gen=datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=False)


validation_generator = generate_batches_data(
                                    dir_path=validation_path,
                                    gen=datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)


Found 18047 images belonging to 10 classes.
Found 4377 images belonging to 10 classes.


In [16]:
#get the lables
nb_train_labels = batch_size * (nb_train_samples // batch_size)
nb_val_labels = batch_size * (nb_val_samples // batch_size)

train_labels = one_hot_encode(train_generator.classes[:nb_train_labels])
validation_labels = one_hot_encode(validation_generator.classes[:nb_val_labels])

print(len(train_labels))
print(len(validation_labels))

17920
4352


In [18]:
#save the train bottleneck features
vgg16_bottleneck_features_train = vgg16_base_model.predict_generator(train_generator, 
                                                                     steps=nb_train_samples // batch_size
                                                                    )
np.save(open('vgg16_bottleneck_features_train.npy', 'wb'),
        vgg16_bottleneck_features_train)

#save the validation bottleneck features
vgg16_bottleneck_features_validation = vgg16_base_model.predict_generator(validation_generator , 
                                                                          steps=nb_val_samples // batch_size
                                                                         )
np.save(open('vgg16_bottleneck_features_validation.npy', 'wb'),
        vgg16_bottleneck_features_validation)

np.save(open('vgg16_train_labels.npy', 'wb'),
        train_labels)

np.save(open('vgg16_validation_labels.npy', 'wb'),
        validation_labels)

In [36]:
#train the top model
train_data = np.load(open('vgg16_bottleneck_features_train.npy','rb'))
validation_data = np.load(open('vgg16_bottleneck_features_validation.npy','rb'))

train_labels = np.load(open('vgg16_train_labels.npy','rb'))
validation_labels = np.load(open('vgg16_validation_labels.npy','rb'))

#we should shuffle the train data before training
train_data,train_labels = shuffle(train_data,train_labels)

top_model = Sequential()
top_model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(class_num, activation='softmax'))

top_model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

top_model.fit(train_data, train_labels,
              epochs=50,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels),
              shuffle=False)
top_model.save_weights('vgg16_top_model_fc.h5')

Train on 17920 samples, validate on 4352 samples
Epoch 1/50
17920/17920 [==============================] - 2s 99us/step - loss: 2.5525 - acc: 0.1237 - val_loss: 2.2022 - val_acc: 0.1337
Epoch 2/50
17920/17920 [==============================] - 1s 71us/step - loss: 2.1216 - acc: 0.1828 - val_loss: 2.2541 - val_acc: 0.2096
Epoch 3/50
17920/17920 [==============================] - 1s 70us/step - loss: 2.0032 - acc: 0.2108 - val_loss: 2.0553 - val_acc: 0.2353
Epoch 4/50
17920/17920 [==============================] - 1s 70us/step - loss: 1.9440 - acc: 0.2353 - val_loss: 2.3422 - val_acc: 0.2436
Epoch 5/50
17920/17920 [==============================] - 1s 70us/step - loss: 1.9198 - acc: 0.2530 - val_loss: 2.2187 - val_acc: 0.2684
Epoch 6/50
17920/17920 [==============================] - 1s 70us/step - loss: 1.8588 - acc: 0.2830 - val_loss: 2.5755 - val_acc: 0.2482
Epoch 7/50
17920/17920 [==============================] - 1s 70us/step - loss: 1.7891 - acc: 0.3137 - val_loss: 2.3690 - val_acc:

### checkpoint

In [36]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg16_base_model.output
# x = Flatten()(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
vgg16_model = Model(inputs=vgg16_base_model.input, outputs=x)

# vgg16_model.summary()


In [37]:
for layer in vgg16_model.layers[:19]:
    layer.trainable = False

for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_12 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 global_average_pooling2d_9 True
20 dense_24 True
21 dropout_11 True
22 dense_25 True


In [39]:

train_datagen = image.ImageDataGenerator(rescale=1./255,
                                         shear_range=0.2,
                                         rotation_range=10,
# #                                          height_shift_range=0.1,
#                                          width_shift_range=0.1,
                                         channel_shift_range=5,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(rescale=1./255,
                                             preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)



Found 18047 images belonging to 10 classes.
Found 4377 images belonging to 10 classes.


In [ ]:
# for layer in vgg16_base_model.layers:
#     layer.trainable = False
    
vgg16_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                      #optimizer=optimizers.Adam(),
                    optimizer='rmsprop',
                    metrics=['accuracy'])

# fine-tune the model
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])

for layer in vgg16_model.layers[15:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),#,decay=1e-6,nesterov=True),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune')]) #,
#                                    EarlyStopping(patience=5)])


vgg16_model.save('vgg16_fine_tuning_model.h5')


Epoch 1/10
140/140 [==============================] - 298s 2s/step - loss: 2.1333 - acc: 0.2444 - val_loss: 2.0260 - val_acc: 0.2626
Epoch 2/10
140/140 [==============================] - 302s 2s/step - loss: 1.7233 - acc: 0.4539 - val_loss: 1.9254 - val_acc: 0.3424
Epoch 3/10
140/140 [==============================] - 299s 2s/step - loss: 1.4173 - acc: 0.5614 - val_loss: 2.1910 - val_acc: 0.2909
Epoch 4/10
140/140 [==============================] - 298s 2s/step - loss: 1.2042 - acc: 0.6324 - val_loss: 2.5579 - val_acc: 0.2412
Epoch 5/10
140/140 [==============================] - 295s 2s/step - loss: 1.0448 - acc: 0.6774 - val_loss: 2.5942 - val_acc: 0.2794
Epoch 1/50
140/140 [==============================] - 296s 2s/step - loss: 0.6989 - acc: 0.7871 - val_loss: 2.3852 - val_acc: 0.3235
Epoch 2/50
140/140 [==============================] - 300s 2s/step - loss: 0.4572 - acc: 0.8631 - val_loss: 2.5499 - val_acc: 0.3747
Epoch 3/50
140/140 [==============================] - 299s 2s/step - 

In [8]:
vgg16_model = load_model('vgg16_fine_tuning_model.h5')

vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-3, momentum=0.9,decay=1e-6,nesterov=True),
                      #optimizer=optimizers.Adam(),
                    metrics=['accuracy'])

for layer in vgg16_model.layers[15:]:
    layer.trainable = True

# fine-tune the model
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=100,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])

vgg16_model.save('vgg16_fine_tuning_model_conv.h5')

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/100
140/140 [==============================] - 297s 2s/step - loss: 1.3192 - acc: 0.5242 - val_loss: 2.8872 - val_acc: 0.2647
Epoch 2/100
140/140 [==============================] - 301s 2s/step - loss: 1.3072 - acc: 0.5304 - val_loss: 2.4069 - val_acc: 0.2975
Epoch 3/100
140/140 [==============================] - 298s 2s/step - loss: 1.3095 - acc: 0.5318 - val_loss: 2.2166 - val_acc: 0.3333
Epoch 4/100
140/140 [==============================] - 296s 2s/step - loss: 1.2934 - acc: 0.5371 - val_loss: 2.6105 - val_acc: 0.2739
Epoch 5/100
140/140 [==============================] - 295s 2s/step - loss: 1.2931 - acc: 0.5338 - val_loss: 2.1102 - val_acc: 0.3107
Epoch 6/100
140/140 [==============================] - 292s 2s/step - loss: 1.2749 - acc: 0.5404 - val_loss: 2.6165 - val_acc: 0.2890
Epoch 7/100
 63/140 [============>.................] - ETA: 2:22 - loss: 1.2838 - acc: 0.5434

KeyboardInterrupt: 

In [ ]:
test_model(model,csv='vgg16_submission.csv')
print('test complete!')